In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import time
%matplotlib inline

In [2]:
df = pd.read_csv('FDIC_data.csv')

/Users/jcheah/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index(['IDRSSD', 'date', 'Financial Institution Name', 'Institution Name ',
       'total_assets', 'defaulter', 'Failure / Assistance ', 'log_TA',
       'NI_to_TA', 'Equity_to_TA', 'NPL_to_TL', 'REO_to_TA', 'ALLL_to_TL',
       'core_deposits_to_TA', 'brokered_deposits_to_TA', 'liquid_assets_to_TA',
       'loss_provision_to_TL', 'ROA', 'NIM', 'assets_growth', 'term_spread',
       'stock_mkt_growth', 'real_gdp_growth', 'unemployment_rate_change',
       'treasury_yield_3m', 'bbb_spread', 'bbb_spread_change',
       'log_TA_plus_1Q', 'NI_to_TA_plus_1Q', 'Equity_to_TA_plus_1Q',
       'NPL_to_TL_plus_1Q', 'REO_to_TA_plus_1Q', 'ALLL_to_TL_plus_1Q',
       'core_deposits_to_TA_plus_1Q', 'brokered_deposits_to_TA_plus_1Q',
       'liquid_assets_to_TA_plus_1Q', 'loss_provision_to_TL_plus_1Q',
       'ROA_plus_1Q', 'NIM_plus_1Q', 'assets_growth_plus_1Q',
       'default_within_1Y', 'Failure', 'max_total_assets'],
      dtype='object')

#### df.drop(columns=['Financial Institution Name','Institution Name ','defaulter','Failure / Assistance ',
                 'default_within_1Y','log_TA_plus_1Q', 'NI_to_TA_plus_1Q', 'Equity_to_TA_plus_1Q',
       'NPL_to_TL_plus_1Q', 'REO_to_TA_plus_1Q', 'ALLL_to_TL_plus_1Q',
       'core_deposits_to_TA_plus_1Q', 'brokered_deposits_to_TA_plus_1Q',
       'liquid_assets_to_TA_plus_1Q', 'loss_provision_to_TL_plus_1Q',
       'ROA_plus_1Q', 'NIM_plus_1Q', 'assets_growth_plus_1Q',],axis=1,inplace=True)

In [4]:
df_failure = df[df.Failure == 1.0]

In [5]:
df_failure['date'] = pd.to_datetime(df_failure['date'])

/Users/jcheah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
train_end_timestamp = pd.Timestamp('2011-01-01')
validation_end_timestamp = pd.Timestamp('2012-01-01')
train_IDRSSD = df_failure[df_failure.date <= train_end_timestamp].IDRSSD.values
validation_IDRSSD = df_failure[(df_failure.date > train_end_timestamp) & (df_failure.date <= validation_end_timestamp)].IDRSSD.values
test_IDRSSD = df_failure[df_failure.date > validation_end_timestamp].IDRSSD.values

In [7]:
print(train_IDRSSD.shape, validation_IDRSSD.shape, test_IDRSSD.shape)

(326,) (66,) (70,)


In [8]:
train_df = df[df.IDRSSD.isin(train_IDRSSD)]
validation_df = df[df.IDRSSD.isin(validation_IDRSSD)]
test_df = df[df.IDRSSD.isin(test_IDRSSD)]

In [9]:
print(train_df.shape, validation_df.shape, test_df.shape)

(10142, 43) (2421, 43) (3058, 43)


In [10]:
train_df.columns

Index(['IDRSSD', 'date', 'Financial Institution Name', 'Institution Name ',
       'total_assets', 'defaulter', 'Failure / Assistance ', 'log_TA',
       'NI_to_TA', 'Equity_to_TA', 'NPL_to_TL', 'REO_to_TA', 'ALLL_to_TL',
       'core_deposits_to_TA', 'brokered_deposits_to_TA', 'liquid_assets_to_TA',
       'loss_provision_to_TL', 'ROA', 'NIM', 'assets_growth', 'term_spread',
       'stock_mkt_growth', 'real_gdp_growth', 'unemployment_rate_change',
       'treasury_yield_3m', 'bbb_spread', 'bbb_spread_change',
       'log_TA_plus_1Q', 'NI_to_TA_plus_1Q', 'Equity_to_TA_plus_1Q',
       'NPL_to_TL_plus_1Q', 'REO_to_TA_plus_1Q', 'ALLL_to_TL_plus_1Q',
       'core_deposits_to_TA_plus_1Q', 'brokered_deposits_to_TA_plus_1Q',
       'liquid_assets_to_TA_plus_1Q', 'loss_provision_to_TL_plus_1Q',
       'ROA_plus_1Q', 'NIM_plus_1Q', 'assets_growth_plus_1Q',
       'default_within_1Y', 'Failure', 'max_total_assets'],
      dtype='object')

In [11]:
def create_data_by_group(group:  pd.DataFrame, freq=4):
    if group.shape[0] < freq:
        return
    group = group.copy()
    label_collection = group['Failure'].values
    group.drop(['IDRSSD', 'date', 'Failure'], axis=1, inplace=True)
    generated_columns = []
    for i in range(freq):
        generated_columns.extend([item + '_' + str(i + 1) for item in group.columns])
    generated_data = []
    for i in range(len(group) - freq + 1):
        tmp = group[i: (i + freq)].values
        tmp = tmp.reshape((1,-1))
        assert tmp.shape[1] == len(generated_columns)
        tmp = pd.DataFrame(tmp, columns=generated_columns)
        if 1. in label_collection[i: (i + freq)]:
            tmp['Failure'] = 1
        else:
            tmp['Failure'] = 0
        generated_data.append(tmp)
    return pd.concat(generated_data)

In [12]:
def create_data(df, freq):
    df = df.copy()
    grouped_df = df.groupby(['IDRSSD'])
    generated_data = []
    for _, group in grouped_df:
        tmp = create_data_by_group(group)
        if tmp is not None:
            generated_data.append(tmp)
    return pd.concat(generated_data)

In [13]:
train_data = create_data(train_df, 4)
validation_data = create_data(validation_df, 4)
test_data = create_data(test_df, 4)

In [21]:
train_data.Failure.value_counts()

0    8838
1     326
Name: Failure, dtype: int64

In [22]:
validation_data.Failure.value_counts()

0    2157
1      66
Name: Failure, dtype: int64

In [25]:
test_data.Failure.value_counts()


0    2778
1      70
Name: Failure, dtype: int64

In [26]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [19]:
train_data.columns

Index(['total_assets_1', 'log_TA_1', 'NI_to_TA_1', 'Equity_to_TA_1',
       'NPL_to_TL_1', 'REO_to_TA_1', 'ALLL_to_TL_1', 'core_deposits_to_TA_1',
       'brokered_deposits_to_TA_1', 'liquid_assets_to_TA_1',
       'loss_provision_to_TL_1', 'ROA_1', 'NIM_1', 'assets_growth_1',
       'term_spread_1', 'stock_mkt_growth_1', 'real_gdp_growth_1',
       'unemployment_rate_change_1', 'treasury_yield_3m_1', 'bbb_spread_1',
       'bbb_spread_change_1', 'max_total_assets_1', 'total_assets_2',
       'log_TA_2', 'NI_to_TA_2', 'Equity_to_TA_2', 'NPL_to_TL_2',
       'REO_to_TA_2', 'ALLL_to_TL_2', 'core_deposits_to_TA_2',
       'brokered_deposits_to_TA_2', 'liquid_assets_to_TA_2',
       'loss_provision_to_TL_2', 'ROA_2', 'NIM_2', 'assets_growth_2',
       'term_spread_2', 'stock_mkt_growth_2', 'real_gdp_growth_2',
       'unemployment_rate_change_2', 'treasury_yield_3m_2', 'bbb_spread_2',
       'bbb_spread_change_2', 'max_total_assets_2', 'total_assets_3',
       'log_TA_3', 'NI_to_TA_3', 'Eq

In [20]:
train_data.to_csv('FDIC_train')
validation_data.to_csv('FDIC_val')
test_data.to_csv('FDIC_test')